In [90]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from math import nan
import json, re, os

headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"}

def render_html(url):
    req = requests.get(url,headers=headers)
    return BeautifulSoup(req.text,"html.parser")

def get_max_page(page):
    max_page = page.find("div",class_="pages").find("span").get_text()
    max_page = [int(d) for d in re.findall("\d+",max_page)][-1]
    return max_page

def get_description(job_url):
    page = render_html(job_url)
    published = page.find("li",id="cp_listed").get_text().split(": ")[-1]
    h3 = page.find("div",class_="single-main").find("h3",class_="description-area")
    description = h3.find_next("p").get_text().strip()
    return published, description

def get_jobs(page):
    jobs = []
    stop_signal = False
    curr_year = datetime.now().year
    for item in page.find_all("div",class_="post-block"):
        a = item.find("a")
        link = a['href']
        title = a['title']
        date, desc = get_description(link)
        date = [d for d in date.split(" ") if ":" not in d]
        year = int(date[-1])
        wage = item.find("p",class_="post-price").get_text().strip()
        if year == curr_year:
            job = {
                "Link": link,
                "Title": title,
                "Jop Description": desc,
                "Wage": wage,
                "Date": " ".join(date)
            }
            jobs.append(job)
        else:
            stop_signal = True
            break
    return jobs, stop_signal

In [91]:
# url = "http://praca-holandia24.pl/ogloszenia/page/3/"
# page = render_html(url)

In [ ]:
# # max page
# max_page = get_max_page(page)
# max_page

In [ ]:
# for item in page.find_all("div",class_="post-block"):
#     a = item.find("a")
#     link = a['href']
#     title = a['title']
#     date, desc = get_description(link)
#     date = [d for d in date.split(" ") if ":" not in d]
#     wage = item.find("p",class_="post-price").get_text().strip()
#     year = date[-1]
#     year = int(year)
#     job = {
#         "Link": link,
#         "Title": title,
#         "Jop Description": desc,
#         "Wage": wage,
#         "Date": " ".join(date)
#     }
#     print(job)
#     print("-"*180)

In [ ]:
# for item in page.find_all("div",class_="post-block"):
#     a = item.find("a")
#     link = a['href']
#     title = a['title']
#     date, desc = get_description(link)
#     date = [d for d in date.split(" ") if ":" not in d]
#     wage = item.find("p",class_="post-price").get_text().strip()
#     year = int(date[-1])
#     job = {
#         "Link": link,
#         "Title": title,
#         "Jop Description": desc,
#         "Wage": wage,
#         "Date": " ".join(date)
#     }

In [93]:
# scrape the first page
url = "http://praca-holandia24.pl/ogloszenia"
page = render_html(url)
max_page = get_max_page(page)
jobs, stop_signal = get_jobs(page)
print(f"1({len(jobs)})",end=",")
    
# iterate and scrape the next page
for i in range(2,max_page+1):
    url = f"http://praca-holandia24.pl/ogloszenia/page/{i}/"
    page = render_html(url)
    new_jobs, stop_signal = get_jobs(page)
    jobs.extend(new_jobs)
    print(f"{i}({len(jobs)})",end=",")
    if stop_signal:
        break
print("\b\b.")

1(10),2(20),3(30),4(40),5(50),6(60),7(70),8(80),9(90),10(100),11(110),12(119).


In [94]:
jobs

[{'Link': 'http://praca-holandia24.pl/ogloszenia/od-zaraz-praca-w-holandii-na-produkcji-kosmetykow-bez-znajomosci-jezyka-w-venlo/',
  'Title': 'Od zaraz praca w Holandii na produkcji kosmetyków bez znajomości języka w Venlo',
  'Jop Description': 'Od zaraz w fabryce kosmetyków z Venlo możliwa do podjęcia stała praca w Holandii na produkcji. Zatrudnienie również bez znajomości języka obcego z zakwaterowaniem zorganizowanym bezpłatnie dla pracowników.',
  'Wage': '12,00/h +zakwaterowanie',
  'Date': '7 lutego 2022'},
 {'Link': 'http://praca-holandia24.pl/ogloszenia/holandia-praca-sezonowa-bez-jezyka-przy-zbiorach-pomidorow-od-zaraz-someren/',
  'Title': 'Holandia praca sezonowa bez języka przy zbiorach pomidorów od zaraz Someren',
  'Jop Description': 'Bez znajomości języka do podjęcia sezonowa praca w Holandii. Zatrudnienie w Someren dla Polaków od zaraz przy zbiorach pomidorów. Zakwaterowanie zorganizuje pracodawca odpłatnie.',
  'Wage': '12,28 € brutto/h',
  'Date': '7 lutego 2022'},
